# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f39f0b00ee0>
├── 'a' --> tensor([[-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840]])
└── 'x' --> <FastTreeValue 0x7f3a1c23ceb0>
    └── 'c' --> tensor([[ 0.3988,  0.1613,  0.9986,  0.7053],
                        [-2.2428, -0.4986, -1.6626, -0.1351],
                        [-0.0078,  0.1313,  2.0593, -1.1287]])

In [4]:
t.a

tensor([[-2.0329,  0.1333,  0.4975],
        [ 0.7507,  1.2132, -1.4840]])

In [5]:
%timeit t.a

65.2 ns ± 0.0793 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f39f0b00ee0>
├── 'a' --> tensor([[-0.1366, -1.2310,  2.2573],
│                   [ 0.2923,  1.3259, -1.2469]])
└── 'x' --> <FastTreeValue 0x7f3a1c23ceb0>
    └── 'c' --> tensor([[ 0.3988,  0.1613,  0.9986,  0.7053],
                        [-2.2428, -0.4986, -1.6626, -0.1351],
                        [-0.0078,  0.1313,  2.0593, -1.1287]])

In [7]:
%timeit t.a = new_value

68.4 ns ± 0.0856 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840]]),
    x: Batch(
           c: tensor([[ 0.3988,  0.1613,  0.9986,  0.7053],
                      [-2.2428, -0.4986, -1.6626, -0.1351],
                      [-0.0078,  0.1313,  2.0593, -1.1287]]),
       ),
)

In [10]:
b.a

tensor([[-2.0329,  0.1333,  0.4975],
        [ 0.7507,  1.2132, -1.4840]])

In [11]:
%timeit b.a

57.8 ns ± 0.143 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4597, -0.2352,  0.9328],
               [-0.3720, -1.0745, -0.5017]]),
    x: Batch(
           c: tensor([[ 0.3988,  0.1613,  0.9986,  0.7053],
                      [-2.2428, -0.4986, -1.6626, -0.1351],
                      [-0.0078,  0.1313,  2.0593, -1.1287]]),
       ),
)

In [13]:
%timeit b.a = new_value

495 ns ± 0.155 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

814 ns ± 0.053 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 473 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 277 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3964525df0>
├── 'a' --> tensor([[[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]],
│           
│                   [[-2.0329,  0.1333,  0.4975],
│                    [ 0.7507,  1.2132, -1.4840]]])
└── 'x' --> <FastTreeValue 0x7f39645259a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.5 µs ± 30 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f3964525fa0>
├── 'a' --> tensor([[-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840],
│                   [-2.0329,  0.1333,  0.4975],
│                   [ 0.7507,  1.2132, -1.4840]])
└── 'x' --> <FastTreeValue 0x7f3a1c23ce50>
    └── 'c' --> tensor([[ 0.3988,  0.1613,  0.9986,  0.7053],
                        [-2.2428, -0.4986, -1.6626, -0.1351],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 39.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]],
       
               [[-2.0329,  0.1333,  0.4975],
                [ 0.7507,  1.2132, -1.4840]]]),
    x: Batch(
           c: tensor([[[ 0.3988,  0.1613,  0.9986,  0.7053],
                       [-2.2428, -0.4986, -1.6626, -0.1351],
                       [-0.0078,  0.1313,  2.0593, -1.1287]],
         

In [26]:
%timeit Batch.stack(batches)

79.1 µs ± 85.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840],
               [-2.0329,  0.1333,  0.4975],
               [ 0.7507,  1.2132, -1.4840]]),
    x: Batch(
           c: tensor([[ 0.3988,  0.1613,  0.9986,  0.7053],
                      [-2.2428, -0.4986, -1.6626, -0.1351],
                      [-0.0078,  0.1313,  2.0593, -1.1287],
                      [ 0.3988,  0.1613,  0.9986,  0.7053],
                      [-2.2428, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

333 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
